In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, models
from torchvision.utils import make_grid
from torchvision import datasets, transforms, models

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import os
from PIL import Image
from IPython.display import display


# ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

In [ ]:
path = '/Users/asadmolayari/Desktop/my_courses/pytorch/Data/CATS_DOGS/test/CAT/10107.jpg'

In [ ]:
with Image.open(path) as m:
    display(m)

In [ ]:
path = '/Users/asadmolayari/Desktop/my_courses/pytorch/Data/CATS_DOGS/'

img_names = []

for folder, subfolders, filenames in os.walk(path):
    for img in filenames:
        img_names.append(folder + '/'+img)

In [ ]:
len(img_names)

In [ ]:
img_names [:3]

In [ ]:
for im in img_names[:3]:
    display(Image.open(im))

In [ ]:
img_sizes = []
rejected = []

for item in img_names:
    try :
        with Image.open(item) as img:
            img_sizes.append(img.size)
            
    except:
        rejected.append(item)
        


In [ ]:
print(len(img_sizes))
len(rejected)

In [ ]:
img_sizes[:4]

In [ ]:
df = pd.DataFrame(img_sizes)

In [ ]:
df.head()

In [ ]:
dog = Image.open(path + 'train/DOG/14.jpg')

In [ ]:
display(dog)

In [ ]:
dog.size

In [ ]:
dog.getpixel((0,0))

In [ ]:
transform = transforms.Compose([
    transforms.ToTensor()
])

im = transform(dog)
print(type(im))
print(im.shape)

In [ ]:
plt.imshow(np.transpose(im.numpy(), (1,2,0)))

In [ ]:
train_transform = transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406],
                        [.229, .224, .225])
    
])

In [ ]:
test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([.485, .456, .406],
                        [.229, .224, .225])

])

In [ ]:
path

In [ ]:
root = path
train_data = datasets.ImageFolder(os.path.join(root, 'train'), transform=train_transform)
test_data = datasets.ImageFolder(os.path.join(root, 'test'), transform=test_transform)

torch.manual_seed(42)

train_loader = DataLoader(train_data, batch_size = 10, shuffle =True)
test_loader = DataLoader(test_data, batch_size= 10)

class_names= train_data.classes

In [ ]:
train_data

In [ ]:
for images, labels in train_loader:
    break

In [ ]:
images.shape

In [ ]:
im = make_grid(images, nrow=5)

inv_normalize = transforms.Normalize(
    mean=[-0.485/0.229, -0.456/0.224, -0.406/0.225],
    std=[1/0.229, 1/0.224, 1/0.225]
)
im_inv = inv_normalize(im)

# Print the images
plt.figure(figsize=(12,4))
plt.imshow(np.transpose(im_inv.numpy(), (1, 2, 0)));

In [ ]:
class ConvolutionNet(nn.Module):
    
    def __init__(self):
        
        super().__init__()
        self.conv1 = nn.Conv2d(3,6,3,1)
        self.conv2 = nn.Conv2d(6,16,3,1)
        self.fc1 = nn.Linear(54*54*16, 120)
        self.fc2 = nn.Linear(120,84)
        self.fc3 = nn.Linear(84,2)
        
    def forward(self, X):
        X = F.relu(self.conv1(X))
        X = F.max_pool2d(X, 2, 2)
        X = F.relu(self.conv2(X))
        X = F.max_pool2d(X, 2, 2)
        X = X.view(-1, 54*54*16)
        X = F.relu(self.fc1(X))
        X = F.relu(self.fc2(X))
        X = self.fc3(X)
        
        return F.log_softmax(X, dim = 1)

In [ ]:
torch.manual_seed(101)
CNNmodel = ConvolutionNet()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(CNNmodel.parameters(), lr = .001)

In [ ]:
CNNmodel

In [ ]:
for p in CNNmodel.parameters():
    print(p.numel())

In [ ]:
import time
start_time = time.time()

epochs = 3

max_trn_batch = 800

max_tst_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    for b, (X_train, y_train) in enumerate(train_loader):
        
        if b == max_trn_batch:
            break
            
        b +=1
        
        y_pred = CNNmodel(X_train)
        loss = criterion(y_pred, y_train)
        
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if b %200 ==0:
            print(f'Epoch {i} LOSS: {loss.item()}')
            
        
    train_losses.append(loss)
    train_correct.append(trn_corr)
    
    with torch.no_grad():
        for b,(X_test, y_test) in enumerate(test_loader):
            
            if b == max_tst_batch:
                break
                
            y_val = CNNmodel(X_test)
            
            predicted = torch.max(y_pred.data, 1)[1]
            barch_corr =(predicted==y_test).sum()
            tst_corr = tst_corr + barch_corr
    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)            

total_time = time.time() - start_time

print(f'Total Time: {total_time/60}')


In [ ]:
plt.plot(train_losses, label='training loss')
plt.plot(test_losses, label='validation loss')
plt.title('Loss at the end of each epoch')
plt.legend();

In [ ]:
plt.plot([t/80 for t in train_correct], label='training accuracy')
plt.plot([t/30 for t in test_correct], label='validation accuracy')
plt.title('Accuracy at the end of each epoch')
plt.legend();

In [ ]:
# applyting AlexNet

In [ ]:
AlexNetmodel = models.alexnet(pretrained=True)

In [ ]:
AlexNetmodel

In [ ]:
for param in AlexNetmodel.parameters():
    param.requires_grad = False

In [ ]:
torch.manual_seed(42)
AlexNetmodel.classifier = nn.Sequential(nn.Linear(9216,1024),
                                        nn.ReLU(),
                                        nn.Dropout(.5),
                                        nn.Linear(1024,2), 
                                        nn.LogSoftmax(dim=1))

In [ ]:
AlexNetmodel

In [ ]:
for param in AlexNetmodel.parameters():
    print(param.numel())

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(AlexNetmodel.classifier.parameters(), lr=.001)

In [ ]:
import time
start_time = time.time()

epochs = 1

max_trn_batch = 800
max_tst_batch = 300

train_losses = []
test_losses = []
train_correct = []
test_correct = []

for i in range(epochs):
    trn_corr = 0
    tst_corr = 0
    
    # Run the training batches
    for b, (X_train, y_train) in enumerate(train_loader):
        if b == max_trn_batch:
            break
        b+=1
        
        # Apply the model
        y_pred = AlexNetmodel(X_train)
        loss = criterion(y_pred, y_train)
 
        # Tally the number of correct predictions
        predicted = torch.max(y_pred.data, 1)[1]
        batch_corr = (predicted == y_train).sum()
        trn_corr += batch_corr
        
        # Update parameters
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print interim results
        if b%200 == 0:
            print(f'epoch: {i:2}  batch: {b:4} [{10*b:6}/8000]  loss: {loss.item():10.8f}  \
accuracy: {trn_corr.item()*100/(10*b):7.3f}%')

    train_losses.append(loss)
    train_correct.append(trn_corr)

    # Run the testing batches
    with torch.no_grad():
        for b, (X_test, y_test) in enumerate(test_loader):
            if b == max_tst_batch:
                break

            # Apply the model
            y_val = AlexNetmodel(X_test)

            # Tally the number of correct predictions
            predicted = torch.max(y_val.data, 1)[1] 
            tst_corr += (predicted == y_test).sum()

    loss = criterion(y_val, y_test)
    test_losses.append(loss)
    test_correct.append(tst_corr)

print(f'\nDuration: {time.time() - start_time:.0f} seconds') # print the time elapsed

In [ ]:
image_index = 1229

im = inv_normalize(test_data[image_index][0])
plt.imshow(np.transpose(im.numpy(), (1,2,0)))

In [ ]:
CNNmodel.eval()

with torch.no_grad():
    new_pred = CNNmodel(test_data[image_index][0].view(1,3,224,224)).argmax()
    
class_names[new_pred.item()]

In [ ]:
AlexNetmodel.eval()

with torch.no_grad():
    new_pred = AlexNetmodel(test_data[image_index][0].view(1,3,224,224)).argmax()
    
class_names[new_pred.item()]